# Data Acquisition for COVID-19 Weather Study

## DS 5559 - Big Data Analytics - Final Project

### Authors
- Aubrey Brockmiller (alb3cb)
- Ian Kloo (ipk8gh)
- Shawn Michanco (sbm5rg)
- Akeem Wells (ajw3rg)

Data acquisition for this project required accessing external data from various sources, cleaning it, and merging.  These tasks are better done outside of Pyspark.  When dealing with weather data specifically, we found an extremely useful and well supported package that only existed in R, `GSODR`, so we used R for the bulk of the data acquisition for this project.

The end result of this notebook is a CSV file that is the source for the work done in `final_project_code.ipynb`.  We did very limited data cleaning here and left most of that effort for the Pyspark code in the other notebook.  Any cleaning steps here were done to ensure we could merge files (e.g., making sure the county-level FIPS codes were correct).

The basic idea is to get everything to the county and day level - i.e., have one row for each county and each day we want to study.

## Reading in Libraries

Here we leverage `data.table` for fast table processing, `rvest` for scraping HTML data, `jsonlite` for processing JSON data, and `GSODR` for accessing the NOAA historical weather data.

In [13]:
library(GSODR)
library(data.table)
library(rvest)
library(jsonlite)

## Acquiring Weather Data

The weather data we want to use is reported at the day level, but not at the county level.  NOAA reports things at the station level where counties can contain any number of stations.  For the purposes of this study, we will start with the county centroids in lat/lon, then look for nearby stations.  We'll average the data from these stations to end up with a decent sense for the data at the county level.

We know that some large counties (e.g., the ones in Alaska) will have significant weather variation at the different stations, but this is something we will have to accept when working at the county level.  If COVID data existed at finer geospatial granularity, it would make sense to use it - but with what we currently have available, this appears to be the best option.

The first task is to get the county centroids.  We'll scrape wikipedia for this:

In [2]:
county_loc <- read_html('https://en.wikipedia.org/wiki/User:Michael_J/County_table') %>%
  html_node('.wikitable') %>%
  html_table() %>%
  data.table()

head(county_loc)

Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AL,1001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,+32.536382°,–86.644490°
2,AL,1003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",+30.659218°,–87.746067°
3,AL,1005,Barbour,Clayton,"27,457","2,291.819",884.876,50.865,19.639,"2,342.684",904.515,+31.870670°,–85.405456°
4,AL,1007,Bibb,Centreville,"22,915","1,612.481",622.582,9.289,3.587,"1,621.770",626.169,+33.015893°,–87.127148°
5,AL,1009,Blount,Oneonta,"57,322","1,669.962",644.776,15.157,5.852,"1,685.119",650.628,+33.977448°,–86.567246°
6,AL,1011,Bullock,Union Springs,"10,914","1,613.057",622.805,6.057,2.338,"1,619.113",625.143,+32.101759°,–85.717261°


The data comes in fairly clean, but we need to process the FIPS codes to account for leading zeros that are dropped on ingest.  This is a common problem with databases storing FIPS codes - they look like numbers but are actually ID's, some with leading zeros.  Here we also process the `Latitude` and `Longitude` columns to get to floats.

In [3]:
options(digits=10)
for(i in 1:nrow(county_loc)){
    
  #fix fips code - if not enough characters, add leading 0
  if(nchar(county_loc$FIPS[i]) != 5){
    county_loc$FIPS[i] <- paste0('0',county_loc$FIPS[i])
  } else{
    county_loc$FIPS[i] <- as.character(county_loc$FIPS[i])
  }
    
  #fix lat lon stuff
  county_loc$Latitude[i] <- as.numeric(substr(county_loc$Latitude[i], 2, nchar(county_loc$Latitude[i]) - 1))
  county_loc$Longitude[i] <- -1*as.numeric(substr(county_loc$Longitude[i], 2, nchar(county_loc$Longitude[i]) - 1))
}
county_loc$Latitude <- as.numeric(county_loc$Latitude)
county_loc$Longitude <- as.numeric(county_loc$Longitude)

head(county_loc)

Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,AL,01001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,32.536382,-86.644490
2,AL,01003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",30.659218,-87.746067
3,AL,01005,Barbour,Clayton,"27,457","2,291.819",884.876,50.865,19.639,"2,342.684",904.515,31.870670,-85.405456
4,AL,01007,Bibb,Centreville,"22,915","1,612.481",622.582,9.289,3.587,"1,621.770",626.169,33.015893,-87.127148
5,AL,01009,Blount,Oneonta,"57,322","1,669.962",644.776,15.157,5.852,"1,685.119",650.628,33.977448,-86.567246
6,AL,01011,Bullock,Union Springs,"10,914","1,613.057",622.805,6.057,2.338,"1,619.113",625.143,32.101759,-85.717261


With the data fairly clean, we can now use `nearest_stations()` provided by the `GSODR` package to get nearby weather stations.  The below outer for-loop runs once for each county.  The interior while-loop grabs the weather stations within 50 miles of the county centroid.  We then attempt to extract data from these nearby stations and store the data in a table.

In some cases there are no weather stations within 50 miles of a county centroid and/or the stations that do exist don't contain some of the records we are interested in (e.g., some stations don't record wind data).  If this happens, the while-loop adds 50 miles to the search radius and continues until we get valid data for that county.  

The end result is a table with a single row for each county/day combination.

In [ ]:
pb <- txtProgressBar(max = nrow(county_loc), style = 3)
out <- list()
for(i in 1:nrow(county_loc)){
  done <- 0
  distance <- 50
  while(done == 0){
    #get list of stations within search radius of county centroid
    stations <- nearest_stations(LAT = county_loc$Latitude[i], LON = county_loc$Longitude[i], distance = distance)
    #extract data for each station captured in the search radius
    w_out <- list()
    for(j in 1:length(stations)){
      tryCatch({
        weather <- suppressMessages(get_GSOD(years = c(2020, 2021), station = stations[j]))
        weather <- weather[, c('YEARMODA', 'TEMP', 'RH', 'VISIB', 'MXSPD','GUST', 'PRCP')]
        w_out[[j]] <- weather
      }, error = function(e){
        NULL 
      })
    }
    weather <- rbindlist(w_out)
    #check if we got valid data for this county, continue with expanded radius if not
    if(nrow(weather) > 0){
      done <- 1
    } else{
      distance <- distance + 50
    }
  }
  #aggregate the station data collected for county
  weather <- weather[, .(TEMP = mean(TEMP, na.rm = TRUE), PRCP = mean(PRCP, na.rm = TRUE),
                         RH = mean(RH, na.rm = TRUE), VISIB = mean(VISIB, na.rm = TRUE),
                         MXSPD = mean(MXSPD, na.rm = TRUE), GUST = mean(GUST, na.rm = TRUE)), by = YEARMODA]
  
  weather$FIPS <- county_loc$FIPS[i]
  out[[i]] <- weather
  setTxtProgressBar(pb, i)
}

final <- rbindlist(out)

In [11]:
head(final)

YEARMODA,TEMP,PRCP,RH,VISIB,MXSPD,GUST,FIPS
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2020-01-01,5.48,0.000,61.66666667,16.10000000,2.48,NaN,01001
2020-01-02,11.06,0.250,79.60000000,14.13333333,6.10,9.30000000,01001
2020-01-03,18.56,21.325,90.90000000,13.13333333,6.50,9.60000000,01001
2020-01-04,14.00,9.400,74.53333333,15.10000000,9.04,13.36666667,01001
2020-01-05,6.10,0.060,58.33333333,15.96666667,4.52,7.70000000,01001
2020-01-06,7.52,0.000,75.36666667,16.10000000,3.52,5.10000000,01001


## Acquiring COVID-19 Data

The COVID-19 data ultimately comes from USA Facts as an authoritative source - but we will access it through an ongoing project that I am a part of in support of the Army's COVID-19 analytic strategy.  A paper describing this project is available here: https://doi.org/10.37266/ISER.2021v9i1.pp2-14.

Data at the county/day level is stored in a github repository that is updated daily.  The below loop accesses the data for each county, sequentially, and merges it with the weather data from above.

In [19]:
fips <- unique(final$FIPS)
cols <- c('countyFIPS','pop','date','deaths','case_count','infect_prob','seven_day_percap',
'r_t','doubling','per_delta','r_t_three','r_t_seven')
pb <- txtProgressBar(max = length(fips), style = 3)
out <- list()
for(i in 1:length(fips)){
  url <- sprintf('https://raw.githubusercontent.com/iankloo/bigmap/master/chart_data/%s.json', fips[i])
  tryCatch({
    js <- data.table(fromJSON(url))
      if(FALSE %in% (cols %in% colnames(js))){
        next
      }
      js <- js[, c('countyFIPS','pop','date','deaths','case_count','infect_prob','seven_day_percap',
                   'r_t','doubling','per_delta','r_t_three','r_t_seven')]
      js[, date := as.Date(date)]
      sub <- final[FIPS == fips[i]]
      out[[i]] <- merge(sub, js, by.x = c('YEARMODA','FIPS'), by.y = c('date','countyFIPS'))
    }, error = function(e){
      NULL 
    })
  setTxtProgressBar(pb, i)
}

final_covid <- rbindlist(out)

  |======================================================================| 100%

In [28]:
tail(final_covid[FIPS == '36027'])

YEARMODA,FIPS,TEMP,PRCP,RH,VISIB,MXSPD,GUST,pop,deaths,case_count,infect_prob,seven_day_percap,r_t,doubling,per_delta,r_t_three,r_t_seven
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
2021-04-30,36027,15.85000000,1.275,62.77500000,16.03333333,9.925000000,17.50000000,294218,441,28640,1.6688,126.7767,0.9917,111,0.0019,0.9655,0.8961
2021-05-01,36027,9.72500000,0.150,46.55000000,16.10000000,7.725000000,15.76666667,294218,441,28698,1.6416,124.0577,1.1043,112,0.0020,1.0252,0.9114
2021-05-02,36027,16.27500000,0.000,41.80000000,16.10000000,6.575000000,11.83333333,294218,442,28744,1.6178,119.6392,0.8811,113,0.0016,0.9924,0.9045
2021-05-03,36027,14.50000000,0.000,73.17500000,15.83333333,3.325000000,NaN,294218,442,28781,1.5193,115.5606,0.7157,114,0.0013,0.9004,0.8931
2021-05-04,36027,14.20000000,12.200,85.95000000,12.63333333,3.850000000,NaN,294218,442,28816,1.4377,114.2010,0.6948,115,0.0012,0.7639,0.8989
2021-05-05,36027,13.53333333,9.250,86.53333333,12.46666667,4.933333333,8.25000000,294218,442,28850,1.3867,107.7432,0.7061,116,0.0012,0.7055,0.8676


## Acquiring Mobility Data

The final data set we used comes from a Google project that is tracking mobility of people at the day/county level using cell phone GPS data.  Their results are stored in a github repo.  Below we access and merge that data with the weather + covid data from above.

In [29]:
mob <- fread('https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-samples.csv', colClasses = c('fips' = 'character'))
mob <- mob[admin_level == 2]
mob$admin_level <- mob$admin1 <- mob$admin2 <- mob$country_code <- NULL

mob_melt <- melt(mob, id.vars = 'fips', variable.name = 'date', value.name = 'm50_index')
mob_melt[, date := as.Date(date)]

final_covid_mob <- merge(final_covid, mob_melt, by.x = c('YEARMODA','FIPS'), by.y = c('date','fips'), all.x = TRUE)

## Final Data Set

After reading data from these three sources and merging everything together, we are left with a final dataset that looks like this:

In [43]:
tail(final_covid_mob[YEARMODA == '2020-09-01'])

YEARMODA,FIPS,TEMP,PRCP,RH,VISIB,MXSPD,GUST,pop,deaths,case_count,infect_prob,seven_day_percap,r_t,doubling,per_delta,r_t_three,r_t_seven,m50_index
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>
2020-09-01,56035,8.3,0.000000000,56.50,16.1,11.850000000,15.45,9831,1,48,0.0000,0.0000,0.0000,40,0.0000,0.0000,0.0000,272
2020-09-01,56037,10.4,0.000000000,47.80,16.1,9.800000000,18.00,42343,2,302,0.2834,16.5317,0.0000,50,0.0000,1.6580,0.8069,1221
2020-09-01,56039,6.9,1.266666667,75.20,14.9,6.666666667,12.10,23464,1,430,0.8950,76.7133,0.0000,44,0.0000,0.5017,0.8049,603
2020-09-01,56041,7.7,0.000000000,56.10,16.1,11.300000000,16.50,20226,2,301,0.8899,88.9944,1.5802,71,0.0101,1.7840,2.6003,633
2020-09-01,56043,14.0,0.000000000,43.00,16.1,4.600000000,NaN,7805,6,108,0.0000,0.0000,0.0000,29,0.0000,0.0000,0.0000,NA
2020-09-01,56045,12.6,0.000000000,44.05,16.1,8.250000000,14.45,6927,0,19,1.0105,86.6176,0.0000,15,0.0000,0.3436,1.3069,NA


The data is still fairly messy and contains more than we will use in our final analysis, but in the interest of using Pyspark for as much of this project as possible, we will stop using R here.  The file created here is the input for `final_project_code.ipynb` - please see that notebook to see our data cleaning, exploratory analysis, modeling, and evaluation processes.

In [44]:
fwrite(final_covid_mob, 'cov_weather_mobility.csv')